# Tarea 4

Hora de entrega: Viernes 14 de marzo, 23:59 pm


Entregar la tarea en este notebook, no usar una adicional a esta.




# Parte 1

- Antes que nada, ¡necesitamos nuestros datos!
1. Ve a la página de microdatos del [INEI](http://iinei.inei.gob.pe/microdatos/).
2. Ve a "Consulta por encuestas" y ve a ENAHO metodología ACTUALIZADA.
3. Elige aquella que dice "Condiciones de Vida y Pobreza - ENAHO" (la que no es panel).
4. Elige el año 2022, y en período, elige la anual.   
5. Descarga los **archivos CSV** del Código módulo 1 (Características de la vivienda y el hogar) y el módulo 34 (Sumarias - Variables Calculadas).

(El nombre del archivo de sumaria es Sumaria-2022).

Esta tarea está pensada en ser desarrollada con los CSV (no los .dta o .sav, que son los archivos de stata ó spss respectivamente).


La ENAHO es una encuesta muy amplia que está distribuida en varios módulos. En esta tarea veremos cómo podemos utilizar información de varios de estos módulos para sacar datos interesantes.

1. Lee ambos archivos csv con pandas y asígnales un nombre para que puedas trabajar con ambos como base de datos. (Si es que sale un error, es debido a que nuestras encuestas están en español y tienen caracteres como ˜ de la ñ o el acento. Por ello, a las opciones de lectura del csv, agrega la siguiente opción ``` encoding = "latin-1" ```.

(No olvides importar pandas 🐼)


En caso tus columnas salgan con el nombre en mayúscula, puedes correr el siguiente comando para cambiarlas a minúsculas:

``` python3
df.columns= df.columns.str.lower()
```
Donde `df` es el nombre del dataframe.   
(En esta clase no ahondamos en el manejo de strings, pero `lower()` es un método de este tipo de variables para convertir el string a minúscula.)

In [ ]:
#Ejercicio 1:
#!pip install pandas
import pandas as pd



sumaria_df = pd.read_csv('Sumaria-2022.csv' , encoding="latin-1")
vivienda_df = pd.read_csv('Enaho01-2022-100.csv' , encoding="latin-1")

sumaria_df.columns = sumaria_df.columns.str.lower()
vivienda_df.columns = vivienda_df.columns.str.lower()

2. - ¿Cuáles son las dimensiones de la sumaria? (o el número de filas y columnas).
   - ¿Cuáles son las dimensiones del módulo de vivienda? (o el número de filas y columnas).

In [ ]:
#ejercicio 2
dimensiones_sumaria = sumaria_df.shape
dimensiones_sumaria
print(f"Dimensiones de la sumaria: {dimensiones_sumaria}")

dimensiones_vivienda = vivienda_df.shape
dimensiones_vivienda
print(f"Dimensiones del módulo de vivienda: {dimensiones_vivienda}")

Dimensiones de la sumaria: (34213, 167)
Dimensiones del módulo de vivienda: (44122, 324)


3. En esta oportunidad no necesitaremos todas las variables de la ENAHO para hacer esta tarea.
- Quedémonos con las variables ``` CONGLOME VIVIENDA HOGAR MIEPERHO PERCEPHO POBREZA GASHOG2D INGHOG2D UBIGEO``` de la sumaria.
- Quedémonos con las variables ``` CONGLOME VIVIENDA HOGAR RESULT NBI1 NBI2 NBI3 NBI4 NBI5 ``` del módulo de vivienda.


 Nota: cada una de las variables es:
- MIEPERHO: número de miembros del hogar.
- PERCEPHO: número de perceptores del hogar (que reciben un ingreso).
- POBREZA: Pobreza monetaria según la línea de pobreza.
- GASHOG2D: Gasto bruto anual del hogar.
- INGHOG2D: Ingreso anual neto total del hogar.
- UBIGEO: Código de 6 dígitos que indica el distrito de pertenencia de la persona.

Necesidades básicas:
- NBI1: vivienda inadecuada.
- NBI2: hacinamiento
- NBI3: vivienda sin servicios higiénicos
- NBI4: hogares con  niños que no asisten a la escuela.
- NBI5: hogares con alta dependencia económica

CONGLOME, VIVIENDA y HOGAR son los identificadores únicos a nivel de hogar (como un key id).

In [ ]:
#ejercicio 3
cols_sumaria = ["conglome", "vivienda", "hogar", "mieperho", "percepho", "pobreza", "gashog2d", "inghog2d", "ubigeo"]
cols_vivienda = ["conglome", "vivienda", "hogar", "result", "nbi1", "nbi2", "nbi3", "nbi4", "nbi5"]
sumaria_df_3 = sumaria_df[cols_sumaria]
vivienda_df_3 = vivienda_df[cols_vivienda]

#por si acaso corroboro con los primeros registros
print("Primeras filas de la sumaria:")
print(sumaria_df_3.head())

print("Primeras filas del módulo de vivienda:")
print(vivienda_df_3.head())

Primeras filas de la sumaria:
   conglome  vivienda  hogar  mieperho  percepho  pobreza      gashog2d  \
0      5030         8     11         2         2        3  36949.523438   
1      5030        17     11         2         2        3  48161.250000   
2      5030        33     11         4         4        3  27575.876953   
3      5030        50     11         3         1        3  16161.042969   
4      5030        59     11        11         7        2  30606.212891   

       inghog2d  ubigeo  
0  34606.441406   10201  
1  82028.218750   10201  
2  31791.029297   10201  
3  24028.867188   10201  
4  46056.109375   10201  
Primeras filas del módulo de vivienda:
   conglome  vivienda  hogar  result nbi1 nbi2 nbi3 nbi4 nbi5
0      5007         3     11       1    0    0    0    0    0
1      5007        12     11       1    0    0    0    0    0
2      5007        22     11       1    0    0    0    0    0
3      5007        31     11       4                         
4      5007   

4. El módulo vivienda incluye todos los hogares que entraron en el marco muestral, incluso aquellos que no participaron en la ENAHO por diversos motivos. Para eliminar estas viviendas, quedémonos con aquellas cuyo ``` RESULT``` sea igual a 1 ó 2. Almacena este resultado para después.

In [ ]:
# 4. Filtrar las viviendas donde RESULT sea 1 o 2
vivienda_filtrado_df = vivienda_df[vivienda_df['result'].isin([1, 2])]

# Verificar resultado
print(vivienda_filtrado_df['result'].value_counts())

result
1    29212
2     5001
Name: count, dtype: int64


5. Queremos calcular una serie de variables (ó columnas nuevas):  
En sumaria:
- Queremos el número de miembros dependientes del hogar . Estos son el número de personas que no perciben un ingreso ó la diferencia entre el total de miembros y el total de perceptores.
- Queremos el ingreso mensual promedio por perceptor del hogar.
- Queremos la diferencia  entre ingreso y gasto del hogar

In [ ]:
sumaria_df.columns

Index(['año', 'mes', 'conglome', 'vivienda', 'hogar', 'ubigeo', 'dominio',
       'estrato', 'mieperho', 'totmieho',
       ...
       'estrsocial', 'factor07', 'ld', 'linpe', 'linea', 'pobreza', 'lineav',
       'pobrezav', 'nconglome', 'sub_conglome'],
      dtype='object', length=167)

In [ ]:
# 5. Verificar y convertir tipos de datos
sumaria_df_5=sumaria_df.copy()
sumaria_df_5[['mieperho', 'percepho', 'inghog2d', 'gashog2d']] = sumaria_df_5[['mieperho', 'percepho', 'inghog2d', 'gashog2d']].apply(pd.to_numeric, errors='coerce')

# Calcular nuevas columnas
sumaria_df_5['miembros_dependientes'] = sumaria_df_5['mieperho'] - sumaria_df_5['percepho']
sumaria_df_5['ingreso_promedio_perceptor'] = sumaria_df_5['inghog2d'] / sumaria_df_5['percepho']
sumaria_df_5['diferencia_ingreso_gasto'] = sumaria_df_5['inghog2d'] - sumaria_df_5['gashog2d']

# Verificar las nuevas columnas
print(sumaria_df_5[['miembros_dependientes', 'ingreso_promedio_perceptor', 'diferencia_ingreso_gasto']].head())

   miembros_dependientes  ingreso_promedio_perceptor  diferencia_ingreso_gasto
0                      0                17303.220703              -2343.082031
1                      0                41014.109375              33866.968750
2                      0                 7947.757324               4215.152344
3                      2                24028.867188               7867.824219
4                      4                 6579.444196              15449.896484


6. La variable de `pobreza`, en la sumaria, está codificada como integers 1, 2 y 3. Esto corresponde a:  
`1`: hogar pobre extremo  
`2`: hogar pobre no extremo  
`3`: hogar no  pobre  

En base a esta variable, crea dos variables más:
1. Una variable que tenga `'pobre extremo'`, `'pobre no extremo'` y `'no pobre'` en vez  de 1, 2, 3.
2. Una variable que tenga `'pobre'` y `'no pobre'` (ó que englobe a los hogares pobres y pobres no extremos).



In [ ]:
# 6. Crear la variable con etiquetas
sumaria_df_6=sumaria_df.copy()
sumaria_df_6['pobreza_etiqueta'] = sumaria_df['pobreza'].map({
    1: 'pobre extremo',
    2: 'pobre no extremo',
    3: 'no pobre'
})

# Crear la variable binaria
sumaria_df_6['pobreza_binaria'] = sumaria_df['pobreza'].apply(lambda x: 'pobre' if x in [1, 2] else 'no pobre')

# Verificar las nuevas columnas
print(sumaria_df_6[['pobreza', 'pobreza_etiqueta', 'pobreza_binaria']].head())

# Verificar conteos de las nuevas columnas
print("Conteo de 'pobreza_etiqueta':\n", sumaria_df['pobreza_etiqueta'].value_counts())
print("Conteo de 'pobreza_binaria':\n", sumaria_df['pobreza_binaria'].value_counts())

   pobreza  pobreza_etiqueta pobreza_binaria
0        3          no pobre        no pobre
1        3          no pobre        no pobre
2        3          no pobre        no pobre
3        3          no pobre        no pobre
4        2  pobre no extremo           pobre
Conteo de 'pobreza_etiqueta':
 pobreza_etiqueta
no pobre            27192
pobre no extremo     5518
pobre extremo        1503
Name: count, dtype: int64
Conteo de 'pobreza_binaria':
 pobreza_binaria
no pobre    27192
pobre        7021
Name: count, dtype: int64


7. Queremos saber el promedio de ingresos de los hogares (inghog2d) por nivel de pobreza. Haz un `groupby` para averiguar esto.


(Por ejemplo, todos los hogares pobres extremos tendrán un único promedio, y así con las 2 categorías de pobreza adicionales).

In [ ]:
# Asegurarse de que los nombres de las columnas estén en minúsculas
sumaria_df.columns = sumaria_df.columns.str.lower()

# Calcular el promedio de ingresos por nivel de pobreza
promedio_ingresos_pobreza = sumaria_df[['inghog2d', 'pobreza']].groupby('pobreza').mean()

# Mostrar el resultado
print(promedio_ingresos_pobreza)

             inghog2d
pobreza              
1        12945.434301
2        20042.135942
3        40513.933057


8. ¡Ahora uniremos los datos!  
Pero antes un check de sanidad: Verifica que el número  de filas de ambas bases de datos son las mismas (acuérdate que filtramos las observaciones por `result`).  La ENAHO recolecta información de estos 34,490 hogares a través de todos sus módulos. Para utilizar estos módulos a la vez, realizaremos una "unión" (merge).   

Une ambas bases con un _inner merge_. Recuerda que el id que identifica a cada hogar se compone de ` conglome vivienda hogar`.  

In [ ]:
# Verificar el número de filas
print("Número de filas en sumaria_df:", len(sumaria_df))
print("Número de filas en vivienda_filtrado_df:", len(vivienda_filtrado_df))

# Unir las bases de datos
merged_df = pd.merge(
    sumaria_df,
    vivienda_filtrado_df,
    on=['conglome', 'vivienda', 'hogar'],
    how='inner'
)

# Verificar el número de filas después del merge
print("Número de filas después del merge:", len(merged_df))

# Verificación
print("Columnas en el dataframe unido:", merged_df.columns)
print("Primeras filas del dataframe unido:\n", merged_df.head())

Número de filas en sumaria_df: 34213
Número de filas en vivienda_filtrado_df: 34213
Número de filas después del merge: 34213
Columnas en el dataframe unido: Index(['año_x', 'mes_x', 'conglome', 'vivienda', 'hogar', 'ubigeo_x',
       'dominio_x', 'estrato_x', 'mieperho', 'totmieho',
       ...
       'nbi4', 'nbi5', 'factor07_y', 'nconglome_y', 'sub_conglome_y',
       'codccpp', 'nomccpp', 'longitud', 'latitud', 'altitud'],
      dtype='object', length=488)
Primeras filas del dataframe unido:
    año_x  mes_x  conglome  vivienda  hogar  ubigeo_x  dominio_x  estrato_x  \
0   2022      1      5030         8     11     10201          7          4   
1   2022      1      5030        17     11     10201          7          4   
2   2022      1      5030        33     11     10201          7          4   
3   2022      1      5030        50     11     10201          7          4   
4   2022      1      5030        59     11     10201          7          4   

   mieperho  totmieho  ...  nbi

9. Haz un ```groupby``` que agrupe (`agg`) los datos por nivel de pobreza  y la nbi1. Queremos hallar el promedio y la mediana de los ingresos (inghog2d) y los gastos (gashog2d) por el producto cartesiano de las categorías de estas 2 variables - pobreza y nbi 1 -.

El producto cartesiano es tan sólo todas las posibles combinaciones entre categorías.
En este caso, tendremos:
- pobre extremo con nbi1
- pobre extremo sin nbi1
- pobre no extremo con nbi1
- pobre no extremo sin nbi1
- no pobre con nbi1
- no pobre sin nbi1

(Este es un  ejemplo de cómo, al cruzar un indicador de pobreza monetaria -pobreza- y pobreza multidimensional-nbis-, podemos seguir observando brechas).

In [ ]:
# Asegurarse de que los nombres de las columnas estén en minúsculas
sumaria_df_8=sumaria_df.copy()
sumaria_df_8.columns = sumaria_df_8.columns.str.lower()

# Realizar el groupby y calcular el promedio y la mediana
resultados = merged_df.groupby(['pobreza', 'nbi1']).agg({
    'inghog2d': ['mean', 'median'],
    'gashog2d': ['mean', 'median']
}).reset_index()

# Renombrar las columnas para mayor claridad
resultados.columns = ['pobreza', 'nbi1', 'inghog2d_mean', 'inghog2d_median', 'gashog2d_mean', 'gashog2d_median']

# Resultado
print(resultados)

   pobreza nbi1  inghog2d_mean  inghog2d_median  gashog2d_mean  \
0        1    0   12789.427903     10092.849609    8384.462050   
1        1    1   13938.827986     11361.211426    9048.719897   
2        2    0   20230.731744     16260.113281   14899.363700   
3        2    1   18634.609608     15313.995117   14865.038216   
4        3    0   41483.639054     30811.552734   32560.459581   
5        3    1   25945.486160     21665.683594   21946.112793   

   gashog2d_median  
0      7918.078613  
1      8502.177246  
2     13943.960938  
3     13673.346680  
4     27251.416016  
5     20046.343750  


10. Crea una variable que nos informe sobre los hogares que tienen al menos dos nbis. Esta tomará el valor de 1 si tiene dos nbis (cualquiera sean estas), 0 en caso contrario (menos de 2).

Calcula el promedio de miembros dependientes por cada categoría de esta nueva variable.

In [ ]:
#Importar db
import pandas as pd
db_enaho=pd.read_csv("Enaho01-2022-100.csv",encoding = "latin-1")
db_enaho.columns=db_enaho.columns.str.lower()
db_sumaria=pd.read_csv("Sumaria-2022.csv",encoding="latin-1")
db_sumaria.columns=db_sumaria.columns.str.lower()

In [ ]:
#Creando dependientes en sumaria
db_sumaria["dependientes"]=db_sumaria['mieperho'] - db_sumaria['percepho']
db_sumaria=db_sumaria[["vivienda","dependientes"]]
db_sumaria.sample(5)

,vivienda,dependientes
8700,58,2
14096,32,0
14877,16,2
6048,3,0
18659,58,1


In [ ]:
#Creando variable de nbis
db_enaho_nbis = db_enaho.copy()
cols_nbi = ["nbi1", "nbi2", "nbi3", "nbi4", "nbi5"]

db_enaho_nbis[cols_nbi] = db_enaho_nbis[cols_nbi].replace([" ", ""], "0").apply(pd.to_numeric)
db_enaho_nbis['nbi_total'] = (db_enaho_nbis[cols_nbi].sum(axis=1) >= 2).astype(int)
#db_enaho_nbis['dependientes'] = db_enaho_nbis['mieperho'] - db_enaho_nbis['percepho']

db_enaho_nbis = db_enaho_nbis[cols_nbi + ['nbi_total',"vivienda"]]

print(db_enaho_nbis.sample(5))


       nbi1  nbi2  nbi3  nbi4  nbi5  nbi_total  vivienda
16942     0     0     0     0     0          0        52
30674     0     0     0     0     0          0        85
25258     0     0     0     0     0          0       109
43903     0     0     0     0     0          0        96
243       0     0     0     0     0          0       147


In [ ]:
#merged db
db_merged = db_enaho_nbis.merge(db_sumaria, on="vivienda", how="inner")
print(db_merged.head())

#Calcular pregunta 10
promedio_dependientes = db_merged.groupby("nbi_total")["dependientes"].mean()
print(promedio_dependientes)

   nbi1  nbi2  nbi3  nbi4  nbi5  nbi_total  vivienda  dependientes
0     0     0     0     0     0          0         3             0
1     0     0     0     0     0          0         3             1
2     0     0     0     0     0          0         3             3
3     0     0     0     0     0          0         3             0
4     0     0     0     0     0          0         3             0
nbi_total
0    1.189566
1    1.192694
Name: dependientes, dtype: float64


Crédito extra:

11. Guarda el dataframe del resultado del ejercicio 9 como un csv. (No es necesario adjuntar este csv, con  que el bloque haya corrido está bien).

In [ ]:
# tu respuesta aquí.
resultados.to_csv("archivo_exportado.csv", index=False)

12. Haz una tabulación cruzada en donde cuentes el número de personas, por nivel de pobreza, que hay en cada departamento. Este nuevo dataframe tendrá como filas a los departamentos y las nuevas columnas, los 3 niveles de pobreza (pobre extremo, pobre y no pobre). Usa `pivot_table`.

*Nota: Los departamentos no necesitan tener nombre, con que estén enumerados del 1 al 25, está bien.
Pista: Puedes utilizar una función que guarde el primer dígito de ubigeo cuando el valor es menor a x, y guardar 2 dígitos cuando el valor es mayor.   
El método de creación de la variable departamento es libre.


In [ ]:
# tu respuesta aquí.
sumaria_df_11=sumaria_df_6.copy()

sumaria_df_11=sumaria_df_11[["vivienda","ubigeo",'pobreza', 'pobreza_etiqueta']]
sumaria_df_11["departamento"] = sumaria_df_11["ubigeo"].astype(str).str[:2].astype(int)
sumaria_df_11

,vivienda,ubigeo,pobreza,pobreza_etiqueta,departamento
0,8,10201,3,no pobre,10
1,17,10201,3,no pobre,10
2,33,10201,3,no pobre,10
3,50,10201,3,no pobre,10
4,59,10201,2,pobre no extremo,10
...,...,...,...,...,...
34208,144,210106,3,no pobre,21
34209,232,210106,3,no pobre,21
34210,233,210106,3,no pobre,21
34211,234,210106,2,pobre no extremo,21


In [ ]:
tabla_pobreza = sumaria_df_11.pivot_table(index="departamento",  # Filas = departamento
                               columns="pobreza_etiqueta",  # Columnas = niveles de pobreza
                               values="vivienda",  # Contamos personas
                               aggfunc="count",  # Contamos registros
                               fill_value=0)  # Reemplazar NaN con 0

display(tabla_pobreza)

pobreza_etiqueta,no pobre,pobre extremo,pobre no extremo
departamento,,,
10,1847,188,493
11,1475,0,45
12,1268,48,205
13,1197,57,290
14,1279,12,121
15,3755,103,694
16,966,136,287
17,570,7,38
18,871,36,82


# Parte 2

1. Escribe una función que toma dos parámetros numéricos, `a` y `b`, y resta ambos números, de tal forma que `c = a - b`. Expresa `c` como el valor a retornar.  


In [10]:
def diferencia(a,b):
    '''
    Objetivo: Restar ambos nros
    Input:
    a: número float
    b: número float
    Output:
    c: número float
    '''
    #Función
    c=a-b
    return c

# Ejemplo
diferencia(7,5)

2

2. Escribe una función que transforme la temperatura de grados celsius a grados fahrenheit. La fórmula es:    
$F = 1.8*C + 32$

In [17]:
def celsius_a_fahrenheit(temp_c):
    '''
    transforma temperatura de celsius a fahrenheit
    Input:
        temp_c:número float
    Ouput:
        retorna: número float
    '''

    #Escribe aquí tu función
    temp_f = 1.8 * temp_c + 32
    return temp_f
celsius_a_fahrenheit(20)

68.0

3.

3.1. Escribe una función que tome como input una lista de números y calcule su promedio.

3.2. Escribe una función que tome como input un diccionario, y que almacene en una lista a todos sus valores.  
Es decir. Si el diccionario es el de la clase:
`ejem_dict =  {"Juana": "gallina", "Martin": "gato", "Carla": "conejo", "Cristina": "gato", "Diego": "perro" } `
Queremos un diccionario = `["gallina", "gato", "conejo","gato", "perro"]`

In [12]:
# EJERCICIO 3.1
def calcular_promedio(lista_nros):
    '''
    Calcula el promedio de una lista de números
    Input:
        lista_nros: lista de números (int o float)
    Output:
        Retorna el promedio como float
    '''
    if len(lista_nros) == 0:
        return 0  # Evita la división entre cero
    return sum(lista_nros) / len(lista_nros)

# Ejemplo
numeros = [17, 20, 23, 26, 29]
print(f"El promedio es: {calcular_promedio(numeros)}")

#EJERCICIO 3.2
def extraer_valores(diccionario):
    '''
    Extrae los valores de un diccionario y los almacena en una lista
    Input:
        diccionario: diccionario con valores de cualquier tipo
    Output:
        Retorna: lista con todos los valores del diccionario
    '''
    return list(diccionario.values())

# Diccionario con nombres y países
personas_países = {
    "Viviana": "Argentina",
    "Jeannethe": "Guatemala",
    "Jesús": "Perú",
    "Winston": "Colombia"
}

# Extracción de valores (países) y almacenamiento en una lista
lista_países = extraer_valores(personas_países)

print(f"Lista de países: {lista_países}")

El promedio es: 23.0
Lista de países: ['Argentina', 'Guatemala', 'Perú', 'Colombia']


4. Tenemos la siguientes función:

In [ ]:
def grado_escolar(edad):

    ''' calcula el grado escolar al que pertences segun tu edad
    Input:
    edad : int
    output: tupla, compuestad de
    grado: string
    esp: especifico que grado

    '''

    if ((edad < 6) | (edad > 17)):
        return "No está en edad escolar"
    #assert edad > 5 & edad < 17
    elif ((edad >=6) & (edad < 12)):
        grado = "Primaria"
        if edad == 6:
            esp = "1er grado"
        elif edad == 7:
            esp = "2do grado"
        elif edad == 8:
            esp = "3er grado"
        elif edad == 9:
            esp = "4to grado"
        elif edad == 10:
            esp = "5to grado"
        else:
            esp = "6to grado"

    else:
        grado = "Secundaria"
        if edad == 12:
            esp = "1er año"
        elif edad == 13:
            esp = "2do año"
        elif edad == 14:
            esp = "3er año"
        elif edad == 15:
            esp = "4to año"
        else:
            esp = "5to año"

    return grado, esp


Asimismo, tenemos a 7 niños, que tienen las siguientes edades:   
    `edades = [2, 4, 6, 7, 10, 15, 16]`  
Queremos saber a qué grados/años van estos niños. Para ello, crea una lista vacía. En dicha lista almacenarás el resultado de dicha función para cada una de las edades de esta lista. Utiliza un for loop .

Pista: Para familiarizarte con la función, puedes utilizarla con edades dentro del rango de edad escolar.

In [18]:
def grado_escolar(edad):

    ''' calcula el grado escolar al que pertences segun tu edad
    Input:
    edad : int
    output: tupla, compuestad de
    grado: string
    esp: especifico que grado

    '''

    if ((edad < 6) | (edad > 17)):
        return "No está en edad escolar"
    #assert edad > 5 & edad < 17
    elif ((edad >=6) & (edad < 12)):
        grado = "Primaria"
        if edad == 6:
            esp = "1er grado"
        elif edad == 7:
            esp = "2do grado"
        elif edad == 8:
            esp = "3er grado"
        elif edad == 9:
            esp = "4to grado"
        elif edad == 10:
            esp = "5to grado"
        else:
            esp = "6to grado"

    else:
        grado = "Secundaria"
        if edad == 12:
            esp = "1er año"
        elif edad == 13:
            esp = "2do año"
        elif edad == 14:
            esp = "3er año"
        elif edad == 15:
            esp = "4to año"
        else:
            esp = "5to año"

    return grado, esp

edades = [2, 4, 6, 7, 10, 15, 16]
lista_de_estudiantes = []
for edad in edades:
    lista_de_estudiantes.append(grado_escolar(edad))
lista_de_estudiantes


['No está en edad escolar',
 'No está en edad escolar',
 ('Primaria', '1er grado'),
 ('Primaria', '2do grado'),
 ('Primaria', '5to grado'),
 ('Secundaria', '4to año'),
 ('Secundaria', '5to año')]